In [2]:
import torchvision
from glob import glob
from random import choice
from torch.utils.data import Dataset
from PIL import Image

import torch
import torch.nn as nn
import numpy as np
import pandas as pd

## Creating a dictionary with (Family) \\\\ (ID) as the key and the path to the images under that one person

In [3]:
import pandas as pd

train_file_path = "./train_relationships.csv"
train_images_path = "./train/"

all_images = glob(train_images_path + "*/*/*.jpg")
train_person_to_images = {}

for x in all_images:

    if x.split("\\")[-3] + "/" + x.split("\\")[-2] not in train_person_to_images:
        train_person_to_images[x.split("\\")[-3] + "/" + x.split("\\")[-2]] = [x]

    else:
        train_person_to_images[x.split("\\")[-3] + "/" + x.split("\\")[-2]].append(x)

train_people = [x.split("\\")[-3] + "/" + x.split("\\")[-2] for x in all_images]
train_people = list(dict.fromkeys(train_people)) # removing the duplicates

relationships = pd.read_csv(train_file_path)
relationships = list(zip(relationships.p1.values, relationships.p2.values))
relationships = [x for x in relationships if x[0] in train_people and x[1] in train_people] #Check if people are in the training dataset

train_person_to_images[relationships[0][0]]

['./train\\F0002\\MID1\\P00009_face3.jpg',
 './train\\F0002\\MID1\\P00010_face4.jpg',
 './train\\F0002\\MID1\\P00011_face1.jpg',
 './train\\F0002\\MID1\\P00012_face2.jpg',
 './train\\F0002\\MID1\\P00013_face2.jpg',
 './train\\F0002\\MID1\\P00014_face2.jpg',
 './train\\F0002\\MID1\\P00015_face2.jpg',
 './train\\F0002\\MID1\\P00016_face2.jpg',
 './train\\F0002\\MID1\\P00017_face3.jpg',
 './train\\F0002\\MID1\\P00018_face1.jpg']